In [ ]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import torch                    # Pytorch module
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import DataLoader # for dataloaders
from PIL import Image           # for checking images
import torch.nn.functional as F
import torchvision.transforms as transforms   # for transforming images into tensors
from torchvision.utils import make_grid       # for data checking
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchsummary import summary              # for getting the summary of our model

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_dir = "/content/drive/MyDrive/cotton data"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"
diseases = os.listdir(train_dir)

In [ ]:
print(diseases)

In [ ]:
print("Total disease classes are: {}".format(len(diseases)))

In [ ]:
import os

def count_images_in_dataset(dataset_path):
    """
    Count the number of images in a dataset.

    Args:
        dataset_path (str): Path to the root directory of the dataset.

    Returns:
        int: Total number of images in the dataset.
    """
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif']  # Add supported image formats
    total_images = 0

    # Walk through the dataset directory
    for root, _, files in os.walk(dataset_path):
        for file in files:
            # Check if the file has a valid image extension
            if any(file.lower().endswith(ext) for ext in image_extensions):
                total_images += 1

    return total_images

# Example usage
if __name__ == "__main__":
    dataset_path = "/content/drive/MyDrive/cotton data"  # Replace with your dataset path
    total_images = count_images_in_dataset(dataset_path)
    print(f"Total number of images in the dataset: {total_images}")


In [ ]:
plants = []
NumberOfDiseases = 0
for plant in diseases:
    parts = plant.split('___')
    if len(parts) > 0 and parts[0] not in plants:  # Check if there's at least one element
        plants.append(parts[0])
    if len(parts) > 1 and parts[1] != 'healthy':  # Check if there's a second element
        NumberOfDiseases += 1

In [ ]:
print("Number of plants: {}".format(len(plants)))

In [ ]:
nums = {}
for disease in diseases:
    nums[disease] = len(os.listdir(train_dir + '/' + disease))

# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

In [ ]:
index = list(range(len(diseases)))
plt.figure(figsize=(20, 5))
# Use diseases as the x-axis labels and corresponding values from nums as heights
plt.bar(index, list(nums.values()), width=0.3)
plt.xlabel('Plants/Diseases', fontsize=10)
plt.ylabel('No of images available', fontsize=10)
# Rotating x-axis labels for better readability
plt.xticks(index, diseases, fontsize=15, rotation=90)
plt.title('Images per each class of plant disease')

In [ ]:
n_train = 0
for value in nums.values():
    n_train += value
print(f"There are {n_train} images for training")

In [ ]:
# datasets for validation and training
train = ImageFolder(train_dir, transform=transforms.ToTensor())
test = ImageFolder(test_dir, transform=transforms.ToTensor())

In [ ]:
img, label = train[0]
print(img.shape, label)

In [ ]:
len(train.classes)

In [ ]:
def show_image(image, label):
    print("Label :" + train.classes[label] + "(" + str(label) + ")")
    plt.imshow(image.permute(1, 2, 0))

In [ ]:
show_image(*train[0])

In [ ]:
show_image(*train[300])

In [ ]:
random_seed = 7
torch.manual_seed(random_seed)

In [ ]:
batch_size = 32

In [ ]:
from torch.utils.data import DataLoader # Import DataLoader from torch.utils.data

# ... (Your existing code) ...

train_dl = DataLoader(train, batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_dl = DataLoader(test, batch_size, num_workers=0, pin_memory=True)

# ... (The rest of your code) ...

In [ ]:
# helper function to show a batch of training instances
def show_batch(data):
    for images, labels in data:
        fig, ax = plt.subplots(figsize=(30, 30))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=8).permute(1, 2, 0))
        break

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a common size
    transforms.ToTensor(),
])

In [ ]:
train = ImageFolder(train_dir, transform=transform)
test = ImageFolder(test_dir, transform=transform)

train_dl = DataLoader(train, batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_dl = DataLoader(test, batch_size, num_workers=0, pin_memory=True)

In [ ]:
!pip install torchvision
import torchvision
from torchvision import transforms

# ... (Your existing code) ...

# Define a transform to resize and convert images to tensors
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a common size
    transforms.ToTensor(),
])

# datasets for validation and training with the resize transform
train = ImageFolder(train_dir, transform=transform)
test = ImageFolder(test_dir, transform=transform)

# ... (The rest of your code) ...

In [ ]:
show_batch(train_dl)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data preprocessing and augmentation
from torchvision import transforms

transform = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Slightly more stable crop range
        transforms.RandomApply([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(degrees=30)
        ], p=0.7),
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.05)
        ], p=0.5),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0)),  # optional: helps generalize
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),

    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),  # More robust than Resize alone
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}


# Load dataset
train_dataset = datasets.ImageFolder("/content/drive/MyDrive/cotton data/train", transform=transform["train"])
val_dataset = datasets.ImageFolder("/content/drive/MyDrive/cotton data/test", transform=transform["val"])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [ ]:
# PyTorch ka example
torch.save(model.state_dict(), 'cotton_model.pth')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save to your Google Drive
torch.save(model.state_dict(), '/content/drive/MyDrive/cotton_model.pth')


In [ ]:
import matplotlib.pyplot as plt

# Epoch range
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(12, 5))

# 🔶 Plot: Training Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, marker='o', color='orange', label='Train Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.grid(True)
plt.legend()

# 🔷 Plot: Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, marker='o', color='green', label='Train Accuracy')
plt.plot(epochs, test_accuracies, marker='s', color='blue', label='Test Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train vs Test Accuracy")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

def compute_confusion_matrix(model, loader, class_names):
    """
    Compute and display the confusion matrix for the model's predictions.

    Args:
        model (torch.nn.Module): Trained model.
        loader (DataLoader): DataLoader for the test dataset.
        class_names (list): List of class names.

    Returns:
        np.ndarray: Confusion matrix.
    """
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # No gradients needed for evaluation
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)  # Move to GPU/CPU
            outputs = model(images)  # Forward pass
            _, predicted = torch.max(outputs, 1)  # Get predicted classes
            all_preds.extend(predicted.cpu().numpy())  # Store predictions
            all_labels.extend(labels.cpu().numpy())  # Store true labels

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds, labels=range(len(class_names)))

    # Display confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
    plt.title("Confusion Matrix")
    plt.show()

    return cm

# Example usage:
if __name__ == "__main__":
    # Define the test dataset and DataLoader
    test_dataset = datasets.ImageFolder(
        "/content/drive/MyDrive/cotton data/test",
        transform=transform["val"]
    )
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # Load the best model
    model.load_state_dict(torch.load("cotton_model.pth"))
    model = model.to(device)

    # Get class names from the dataset
    class_names = test_dataset.classes  # List of class names

    # Compute and display the confusion matrix
    confusion_matrix_result = compute_confusion_matrix(model, test_loader, class_names)

In [ ]:
import matplotlib.pyplot as plt

# Example data: Learning rate values per batch
# Replace these with actual learning rates if you are using a learning rate scheduler
batch_numbers = list(range(1, 51))  # Example: 100 batches
learning_rates = [0.001 * (0.95 ** (batch // 10)) for batch in batch_numbers]  # Example LR decay

# Plot Learning Rate vs. Batch Number
plt.figure(figsize=(10, 5))
plt.plot(batch_numbers, learning_rates, color='blue', marker='o', markersize=3, label='Learning Rate')

plt.title('Learning Rate vs. Batch Number')
plt.xlabel('Batch Number')
plt.ylabel('Learning Rate')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
test_dir = "/content/drive/MyDrive/cotton data/test"
test = ImageFolder(test_dir, transform=transforms.ToTensor())

In [ ]:
test_images = sorted(os.listdir(test_dir)) # since images in test folder are in alphabetical order
test_images

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
def predict_image(img, model):
    """Converts image to array and return the predicted class
        with highest probability"""
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label

    return train_dataset.classes[preds[0].item()]

In [ ]:
!pip install flask flask-ngrok


In [ ]:
!pip install pyngrok

In [ ]:
# Install required libraries (run this in Colab cell)
# !pip install flask flask-ngrok torch torchvision pyngrok

from flask import Flask, request, render_template_string, jsonify
from pyngrok import ngrok, conf
from torchvision import transforms, models
from PIL import Image
import torch
import torch.nn as nn

# Define the EfficientNetV2 model class (same as in your training script)
class EfficientNetV2Model(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetV2Model, self).__init__()
        self.base_model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

# Set the number of classes (update this based on your dataset)
num_classes = 7
model = EfficientNetV2Model(num_classes)

# Load your trained model file
state_dict = torch.load('/content/cotton_model.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.eval()  # Set the model to evaluation mode

# Define image preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pretrained models
])

# Define class labels (update based on your dataset)
class_labels = ['Aphids','Army worm','Bacterial blight','Healthy','Powdery mildew','Target spot','Unknown']

# Configure ngrok
conf.get_default().auth_token = "2sAaR9tq50mYwCQMZAuZp214SwT_5BFQ8WoeaG8okfuiVmoL2"  # Replace with your ngrok token
ngrok_tunnel = ngrok.connect(5000)
print(' * Public URL:', ngrok_tunnel.public_url)

# Initialize Flask app
app = Flask(__name__)

@app.route('/')
def index():
    return render_template_string()

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'prediction': 'Undefined'}), 400

    file = request.files['file']
    try:
        image = Image.open(file.stream).convert('RGB')
        image = transform(image).unsqueeze(0)

        with torch.no_grad():
            outputs = model(image)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            confidence, predicted = torch.max(probabilities, 1)

            confidence_score = confidence.item()
            predicted_label = class_labels[predicted.item()]

            print(f"[DEBUG] Predict: {predicted_label} | Confidence: {confidence_score:.2f}")

            return jsonify({'prediction': predicted_label})

    except Exception as e:
        return jsonify({'prediction': 'Undefined', 'error': str(e)}), 500


# Run the app
app.run(port=5000)
